In [ ]:
%load_ext autoreload
%autoreload 2

import os,sys,warnings
import h5py, pandas
import numpy as np
np.seterr(invalid=['ignore','warn'][0])
import math,cmath,pickle
from matplotlib.backends.backend_pdf import PdfPages
from scipy.optimize import curve_fit,fsolve
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.style.use('default')
mpl.rcParams['figure.facecolor'] = 'white'
mpl.rcParams['figure.titlesize'] = 20
mpl.rcParams['figure.figsize'] = [6.4*1.2,4.8*1.2]
mpl.rcParams['axes.labelsize'] = 30
mpl.rcParams['axes.titlesize'] = 30
mpl.rcParams['lines.marker'] = 's'
mpl.rcParams['lines.linestyle'] = ''
mpl.rcParams['lines.markersize'] = 12
mpl.rcParams['errorbar.capsize'] = 12
mpl.rcParams['xtick.labelsize'] = mpl.rcParams['ytick.labelsize'] = 22
mpl.rcParams['xtick.major.size'] = mpl.rcParams['ytick.major.size'] = 10
mpl.rcParams['xtick.top']=mpl.rcParams['ytick.right']=True
mpl.rcParams['xtick.direction']=mpl.rcParams['ytick.direction']='in'
mpl.rcParams['legend.fontsize'] = 24
plt.rcParams["font.family"] = "serif"
plt.rcParams["mathtext.fontset"] = "dejavuserif"

# mpl.rcParams.update({"axes.grid" : True})
import util as yu
yu.flag_fast=False

def fsolve2(func,x0):
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        res=fsolve(func, x0)[0]
    return res if res!=x0 else np.NaN

In [ ]:
enss=['a','b','c','d']
# enss=['a','b']
ens2full={'a':'cA2.09.48','b':'B64','c':'C80','d':'D96'}
ens2label={'a':'A48','b':'B64','c':'C80','d':'D96'}
ens2a={'a':0.0938,'b':0.07957,'c':0.06821,'d':0.05692} # fm
ens2N={'a':48,'b':64,'c':80,'d':96}

hbarc = 1/197.3
ens2aInv={ens:1/(ens2a[ens]*hbarc) for ens in enss} # MeV

print([135*ens2a[ens]*ens2N[ens]*hbarc for ens in enss]) # mpi*L
print(np.exp(np.array([135*ens2a[ens]*ens2N[ens]*hbarc for ens in enss])*(-1))) # e^{-mpi*L}

t=np.array([861243,1668373,2858890])
tN=np.array([ens2N[ens]**3 for ens in ['b','c','d']])
print(tN/tN[1])
t=t/np.array(tN)
print(t/t[1])
    
print()
data={}
path='/p/project/pines/li47/code/projectData/NST_e-scattering/pion2pt_more/'
for ens in enss:
    data[ens]={}
    with h5py.File(path+ens2full[ens]+'.h5') as f:
        for ky in f['data'].keys():
            data[ens][ky]=np.complex128(f['data'][ky][:])
    print(ens,len(data[ens]['vev']),len(yu.jackknife(data[ens]['vev'])))

In [ ]:
def block(dat,d):
    n=len(dat)
    n=n//d*d
    dat=dat[:n]
    shape=dat.shape
    shape_new=[d,n//d]
    
# auto correlation
def run(ens):
    if True: # delete d comparison
        fig, axs = yu.getFigAxs(1,1,scale=1,sharex=True,sharey=True)
        xUnit=1; yUnit=1
        
        time=20
        print(time*ens2a[ens])
        
        baseErr=None
        for i,d in enumerate(range(1,64)):
            timeT=data[ens]['piC'].shape[1]
            tfunc_C=lambda m,t:np.exp(-m*t)+np.exp(-m*(timeT-t))
            tau=1
            def C2pt2meff(C2pt):
                t=C2pt # dim=time
                t=(np.roll(t,-tau,axis=0))/(t)
                t=np.array([fsolve2(lambda m:((tfunc_C(m,xt+tau))/(tfunc_C(m,xt)))-ele, 0.0593) for xt,ele in enumerate(t)])
                return t
            
            dat=data[ens]['pi0_conn'][:,:,0]+data[ens]['pi0_disc'][:,:,0]
            dat_vev=data[ens]['vev']
            C2pt0_0sub_jk=yu.jackknife(dat,d=d)
            vev_jk=yu.jackknife(dat_vev,d=d)
            C2pt_jk=np.real(C2pt0_0sub_jk-vev_jk[:,None]**2)
            C2pt_jk=C2pt_jk[:,time:time+1]
            # yu.jackmap(C2pt2meff,C2pt_jk)
            mean,err=yu.jackme(C2pt_jk)
            
            if baseErr is None:
                baseErr=err
            
            irow=0; icol=0
            plt_x=d*xUnit; plt_y=err[0]/baseErr[0]*yUnit; plt_yerr=0
            axs[irow,icol].errorbar(plt_x,plt_y,plt_yerr,color='r')
                
        baseErr=None
        for i,d in enumerate(range(1,64)):
            timeT=data[ens]['piC'].shape[1]
            tfunc_C=lambda m,t:np.exp(-m*t)+np.exp(-m*(timeT-t))
            tau=1
            def C2pt2meff(C2pt):
                t=C2pt # dim=time
                t=(np.roll(t,-tau,axis=0))/(t)
                t=np.array([fsolve2(lambda m:((tfunc_C(m,time+tau))/(tfunc_C(m,time)))-ele, 0.0593) for xt,ele in enumerate(t)])
                return t
            
            dat=data[ens]['pi0_conn'][:,:,0]+data[ens]['pi0_disc'][:,:,0]
            dat_vev=data[ens]['vev']
            C2pt0_0sub_jk=yu.jackknife(dat,d=d)
            vev_jk=yu.jackknife(dat_vev,d=d)
            C2pt_jk=np.real(C2pt0_0sub_jk-vev_jk[:,None]**2)
            C2pt_jk=C2pt_jk[:,time:time+2]
            m_jk=yu.jackmap(C2pt2meff,C2pt_jk)
            mean,err=yu.jackme(m_jk)
            
            if baseErr is None:
                baseErr=err
            
            irow=0; icol=0
            # print(d,mean[0],err[0])
            plt_x=(d+0.2)*xUnit; plt_y=err[0]/baseErr[0]*yUnit; plt_yerr=0
            axs[irow,icol].errorbar(plt_x,plt_y,plt_yerr,color='g')
        
    # dat_pi0=data[ens]['pi0_conn'][:,:,0]+data[ens]['pi0_disc'][:,:,0]
    # dat_vev=data[ens]['vev']
    # dat=[dat_pi0,dat_vev]
    # def func(dat):
    #     t_pi0,t_vev=dat
        
run('b')

In [ ]:
# ens='c'
# ens2={'a':'cA2.09.48','b':'B64','c':'C80','d':'D96'}[ens]

# ky='pi0_conn'
# ky='piC'

# ens='c'
# t=np.real(data[ens]['pi0_disc'][:,:,0])
# tvev=np.real(data[ens]['vev'])
# t_jk=yu.jackknife(t)
# tvev_jk=yu.jackknife(tvev)
# t_jk=t_jk-tvev_jk[:,None]**2
# print(t_jk.shape)
# t=np.sum(t_jk,axis=0)[None,:] - (len(t_jk)-1)*t_jk
# print(t_jk[:10,:4])

# ky='pi0_disc'

# path=f'/p/project/pines/li47/code/projectData/NST_e-scattering/pion2pt/{ens2}.h5'
# with h5py.File(path) as f:
#     print(f.keys())
#     print(f['data'].keys())
#     print(f['cfgs'][:10])
#     t=f['data'][ky][:,:,0]
#     t_jk=yu.jackknife(t)
#     print(t_jk[:10,:4])

In [ ]:
ens2fitmins={'a':range(4,38), 'b':range(4,45), 'c':range(4,45), 'd':range(4,48)}

ens2mC_jk={}
ens2m0_jk={}
ens2m0d_jk={}
ens2fitC={}
ens2fit0_0c={}
ens2fit0={}

def run(ens):
    C2pt0_0sub_jk=yu.jackknife(data[ens]['pi0_conn'][:,:,0]+data[ens]['pi0_disc'][:,:,0])
    vev_jk=yu.jackknife(data[ens]['vev'])
    C2pt0_jk=np.real(C2pt0_0sub_jk-vev_jk[:,None]**2)
    C2ptC_jk=np.real(yu.jackknife(data[ens]['piC'][:,:,0]))
    
    # symmetrize
    func=lambda t:(t+np.roll(np.flip(t,axis=1),1,axis=1))/2
    C2ptC_jk=func(C2ptC_jk)
    C2pt0_jk=func(C2pt0_jk)
    
    # effective masses
    timeT=data[ens]['piC'].shape[1]
    tfunc_C=lambda m,t:np.exp(-m*t)+np.exp(-m*(timeT-t))
    ### effective mass from C(t+tau)/C(t)
    tau=1
    def C2pt2meff(C2pt):
        t=C2pt # dim=time
        t=(np.roll(t,-tau,axis=0))/(t)
        t=np.array([fsolve2(lambda m:((tfunc_C(m,xt+tau))/(tfunc_C(m,xt)))-ele, 0.0593) for xt,ele in enumerate(t)])
        return t
    ens2mC_jk[ens]=mC_jk=yu.jackmap(C2pt2meff,C2ptC_jk)
    ens2m0_jk[ens]=m0_jk=yu.jackmap(C2pt2meff,C2pt0_jk)
    ### effective mass from [C(t+tau)-C(t+tau+td)]/[C(t)-C(t+td)]
    (tau,td)={'a':(4,4),'b':(13,13),'c':(15,15),'d':(18,18)}[ens]
    def C2pt2meff(C2pt):
        t=C2pt # dim=time
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            t=(np.roll(t,-tau,axis=0)-np.roll(t,-td-tau,axis=0))/(t-np.roll(t,-td,axis=0))
        t=np.array([fsolve2(lambda m:((tfunc_C(m,xt+tau)-tfunc_C(m,xt+tau+td))/(tfunc_C(m,xt)-tfunc_C(m,xt+td)))-ele, 0.0593) for xt,ele in enumerate(t)])
        return t
    ens2m0d_jk[ens]=m0_jk=yu.jackmap(C2pt2meff,C2pt0_jk)
    
    C2ptC_mean,C2ptC_err,C2ptC_cov=yu.jackmec(C2ptC_jk)
    C2pt0_mean,C2pt0_err,C2pt0_cov=yu.jackmec(C2pt0_jk)
    C2pt0_rela=np.abs(C2pt0_err/C2pt0_mean)
    print(np.max(C2pt0_rela))
    
    if False:
        fitmax=timeT//2+1
        ### piC
        pars0=[0.06,C2ptC_jk[0,1]]; ens2fitC[ens]={}
        for fitmin in ens2fitmins[ens]:
            print(ens,fitmin,end='        \r')
            ts=np.arange(fitmin,fitmax)
            fitfunc=lambda pars: pars[1]*(np.exp(-pars[0]*ts)+np.exp(-pars[0]*(timeT-ts))) # + pars[2] # pars=[m,c,const]
            ens2fitC[ens][fitmin]=res=yu.jackfit(fitfunc,C2ptC_jk[:,ts],pars0)
            if res is not None:
                pars0=res[0][0]
        ### pi0
        pars0=[0.06,C2pt0_jk[0,1]]; ens2fit0_0c[ens]={}
        for fitmin in ens2fitmins[ens]:
            print(ens,fitmin,end='        \r')
            ts=np.arange(fitmin,fitmax)
            fitfunc=lambda pars: pars[1]*(np.exp(-pars[0]*ts)+np.exp(-pars[0]*(timeT-ts))) # + pars[2] # pars=[m,c,const]
            ens2fit0_0c[ens][fitmin]=res=yu.jackfit(fitfunc,C2pt0_jk[:,ts],pars0)
            if res is not None:
                pars0=res[0][0]
        ### pi0
        pars0=[0.06,C2pt0_jk[0,1],0]; ens2fit0[ens]={}
        for fitmin in ens2fitmins[ens]:
            print(ens,fitmin,end='        \r')
            ts=np.arange(fitmin,fitmax)
            fitfunc=lambda pars: pars[1]*(np.exp(-pars[0]*ts)+np.exp(-pars[0]*(timeT-ts))) + pars[2] # pars=[m,c,const]
            ens2fit0[ens][fitmin]=res=yu.jackfit(fitfunc,C2pt0_jk[:,ts],pars0)
            if res is not None:
                pars0=res[0][0]
    
for ens in enss:
    run(ens)

In [ ]:
ens2fitmins_plt={'a':range(4,38-5), 'b':range(4,45-8), 'c':range(4,45-5), 'd':range(4,48-7)}
ens2chi2jump={'a':3,'b':3,'c':3,'d':4}
ens2mass_jk={}

fig, axs = yu.getFigAxs(4,2,scale=1,sharey=True,sharex='col')
yu.addRowHeader(axs,[ens2label[ens] for ens in enss])
yu.addColHeader(axs,['mEff','fits'])
    
axs[0,0].set_xlim([0,6]); axs[0,1].set_xlim([0,3.7])
axs[0,0].set_ylim([0,200])

for irow,ens in enumerate(enss):
    print(ens)
    ens2mass_jk[ens]={}
    mC_jk=ens2mC_jk[ens]
    m0_jk=ens2m0_jk[ens]
    m0d_jk=ens2m0d_jk[ens]
    
    mC_mean,mC_err,mC_cov=yu.jackmec(mC_jk)
    m0_mean,m0_err,m0_cov=yu.jackmec(m0_jk)
    m0d_mean,m0d_err,m0d_cov=yu.jackmec(m0d_jk)
    
    icol=0; xUnit=ens2a[ens]; yUnit=ens2aInv[ens]
    tmean,terr=mC_mean,mC_err
    axs[irow,icol].errorbar((np.arange(len(tmean)))*xUnit,tmean*yUnit,terr*yUnit,color='r',label='piC')
    tmean,terr=m0_mean,m0_err
    axs[irow,icol].errorbar((np.arange(len(tmean)))*xUnit,tmean*yUnit,terr*yUnit,color='b',label='pi0_r')
    # tmean,terr=m0d_mean,m0d_err
    # axs[irow,icol].errorbar((np.arange(len(tmean)))*xUnit,tmean*yUnit,terr*yUnit,color='purple',label='pi0_rd')
    
    if False:
        icol=1
        fits=[]; fitLs=[]
        for fitmin in ens2fitmins_plt[ens]:
            color='r'
            pars_jk,chi2_jk,Ndof=ens2fitC[ens][fitmin]
            chi2R=np.mean(chi2_jk)/Ndof
            if chi2R>100:
                continue
            fits.append([pars_jk,chi2_jk,Ndof]); fitLs.append(fitmin)
            pars_mean,pars_err=yu.jackme(pars_jk)
            plt_x=fitmin*xUnit; plt_y=pars_mean[0]*yUnit; plt_yerr=pars_err[0]*yUnit
            axs[irow,icol].errorbar(plt_x,plt_y,plt_yerr,color=color)
            if chi2R>10 or fitmin%ens2chi2jump[ens] != 0:
                continue
            ylim=axs[irow,icol].get_ylim()
            axs[irow,icol].annotate("%0.1f" %chi2R,(plt_x,ylim[1]),color=color,ha='center')
        pars_jk,props_jk=yu.jackMA(fits)
        ens2mass_jk[ens]['mC']=mC_MA_jk=pars_jk[:,:1]

        
        tmean,terr=yu.jackme(mC_MA_jk)
        plt_y=tmean*yUnit; plt_yerr=terr*yUnit
        axs[irow,icol].fill_between([0,10],plt_y-plt_yerr,plt_y+plt_yerr,color=color,alpha=1)

        fits=[]
        for fitmin in ens2fitmins_plt[ens]:
            color='b'
            pars_jk,chi2_jk,Ndof=ens2fit0_0c[ens][fitmin]
            chi2R=np.mean(chi2_jk)/Ndof
            if chi2R>100:
                continue
            fits.append([pars_jk,chi2_jk,Ndof])
            pars_mean,pars_err=yu.jackme(pars_jk)
            plt_x=fitmin*xUnit; plt_y=pars_mean[0]*yUnit; plt_yerr=pars_err[0]*yUnit
            axs[irow,icol].errorbar(plt_x,plt_y,plt_yerr,color=color)
            if chi2R>10 or fitmin%ens2chi2jump[ens] != 0:
                continue
            ylim=axs[irow,icol].get_ylim()
            axs[irow,icol].annotate("%0.1f" %chi2R,(plt_x,ylim[0]),color=color,ha='center')
        pars_jk,props_jk=yu.jackMA(fits)
        ens2mass_jk[ens]['m0_0c']=m0_0c_MA_jk=pars_jk[:,:1]
        
        tmean,terr=yu.jackme(m0_0c_MA_jk)
        plt_y=tmean*yUnit; plt_yerr=terr*yUnit
        axs[irow,icol].fill_between([0,10],plt_y-plt_yerr,plt_y+plt_yerr,color=color,alpha=0.2)
        
        fits=[]
        for fitmin in ens2fitmins_plt[ens]:
            color='purple'
            pars_jk,chi2_jk,Ndof=ens2fit0[ens][fitmin]
            chi2R=np.mean(chi2_jk)/Ndof
            if chi2R>100:
                continue
            fits.append([pars_jk,chi2_jk,Ndof])
            pars_mean,pars_err=yu.jackme(pars_jk)
            plt_x=fitmin*xUnit; plt_y=pars_mean[0]*yUnit; plt_yerr=pars_err[0]*yUnit
            axs[irow,icol].errorbar(plt_x,plt_y,plt_yerr,color=color)
            if chi2R>10 or fitmin%ens2chi2jump[ens] != 0:
                continue
            ylim=axs[irow,icol].get_ylim()
            axs[irow,icol].annotate("%0.1f" %chi2R,(plt_x,ylim[0]+20),color=color,ha='center')
        pars_jk,props_jk=yu.jackMA(fits)
        ens2mass_jk[ens]['m0']=m0_MA_jk=pars_jk[:,:1]
        ens2mass_jk[ens]['pars']=pars_jk
        
        tmean,terr=yu.jackme(m0_MA_jk)
        plt_y=tmean*yUnit; plt_yerr=terr*yUnit
        axs[irow,icol].fill_between([0,10],plt_y-plt_yerr,plt_y+plt_yerr,color=color,alpha=0.2)

axs[0,0].legend(fontsize=10,loc='upper center')

plt.tight_layout()
plt.savefig(f'fig/fits.pdf')
plt.close()

In [ ]:
cases=['mC','m0','mC-m0','c2','rt']

def get(ens,case):
    if case is 'rt':
        timeT=2*ens2N['a']
        time=timeT/2
        fitfunc=lambda pars: pars[1]*(np.exp(-pars[0]*time)+np.exp(-pars[0]*(timeT-time))) + pars[2] 
        func=lambda pars: [pars[2]/(fitfunc(pars)-pars[2])]
        rt_jk=yu.jackmap(func,ens2mass_jk[ens]['pars'])
        tmean,terr=yu.jackme(rt_jk)
        return yu.un2str(tmean[0],terr[0],precision=3)
    
    func=case2func={'mC': lambda dic:dic['mC']*ens2aInv[ens],
               'm0': lambda dic:dic['m0']*ens2aInv[ens],
               'mC-m0': lambda dic:(dic['mC']-dic['m0'])*ens2aInv[ens],
               'c2': lambda dic:(dic['m0']**2-dic['mC']**2)*(ens2aInv[ens]**4)*1e-12,
        }[case]
    tmean,terr=yu.jackme(func(ens2mass_jk[ens]))
    return yu.un2str(tmean[0],terr[0],precision=3)

tab=[[get(ens,case) for ens in enss] for case in cases]
t=pandas.DataFrame(tab, cases, [ens2label[ens] for ens in enss])
print(t)

fig, axs = yu.getFigAxs(1,1)
xlim=[-1,4]
axs[0,0].set_xticks([0,1,2,3][:len(enss)])
axs[0,0].set_xticklabels([ens2label[ens] for ens in enss])
axs[0,0].set_xlim(xlim)
axs[0,0].set_xlabel('ens')
axs[0,0].set_ylabel('c2')
axs[0,0].set_ylim([-0.1,0.02])
axs[0,0].hlines(0,xlim[0],xlim[1],color='black',linestyle='--')

for i,ens in enumerate(enss):
    color='r' if ens!='a' else 'b'
    func=lambda dic:(dic['m0']**2-dic['mC']**2)*(ens2aInv[ens]**4)*1e-12
    tmean,terr=yu.jackme(func(ens2mass_jk[ens]))
    plt_x=[i]; plt_y=tmean; plt_yerr=terr
    axs[0,0].errorbar(plt_x,plt_y,plt_yerr,color=color)

plt.tight_layout()
plt.savefig(f'fig/c2.pdf')
# plt.close()

In [ ]:
# test jackknife + modelAvg
ens2chi2jump={'a':3,'b':3,'c':3,'d':4}
ens2mass_jk={}

fig, axs = yu.getFigAxs(4,2,scale=1,sharey=True,sharex='col')
yu.addRowHeader(axs,[ens2label[ens] for ens in enss])
yu.addColHeader(axs,['mEff','fits'])
    
axs[0,0].set_xlim([0,6]); axs[0,1].set_xlim([0,3.7*2])
axs[0,0].set_ylim([0,200])

for irow,ens in enumerate(enss):
    print(ens)
    ens2mass_jk[ens]={}
    mC_jk=ens2mC_jk[ens]
    m0_jk=ens2m0_jk[ens]
    m0d_jk=ens2m0d_jk[ens]
    
    mC_mean,mC_err,mC_cov=yu.jackmec(mC_jk)
    m0_mean,m0_err,m0_cov=yu.jackmec(m0_jk)
    m0d_mean,m0d_err,m0d_cov=yu.jackmec(m0d_jk)
    
    icol=0; xUnit=ens2a[ens]; yUnit=ens2aInv[ens]
    tmean,terr=mC_mean,mC_err
    axs[irow,icol].errorbar((np.arange(len(tmean)))*xUnit,tmean*yUnit,terr*yUnit,color='r',label='piC')
    tmean,terr=m0_mean,m0_err
    axs[irow,icol].errorbar((np.arange(len(tmean)))*xUnit,tmean*yUnit,terr*yUnit,color='b',label='pi0_r')
    tmean,terr=m0d_mean,m0d_err
    axs[irow,icol].errorbar((np.arange(len(tmean)))*xUnit,tmean*yUnit,terr*yUnit,color='purple',label='pi0_rd')
    
    icol=1
    fits=[]; fitLs=[]
    for fitmin in ens2fitmins_plt[ens]:
        color='r'
        pars_jk,chi2_jk,Ndof=ens2fitC[ens][fitmin]
        chi2R=np.mean(chi2_jk)/Ndof
        if chi2R>100:
            continue
        fits.append([pars_jk,chi2_jk,Ndof]); fitLs.append(fitmin)
        pars_mean,pars_err=yu.jackme(pars_jk)
        plt_x=fitmin*xUnit; plt_y=pars_mean[0]*yUnit; plt_yerr=pars_err[0]*yUnit
        axs[irow,icol].errorbar(plt_x,plt_y,plt_yerr,color=color)
        if chi2R>10 or fitmin%ens2chi2jump[ens] != 0:
            continue
        ylim=axs[irow,icol].get_ylim()
        axs[irow,icol].annotate("%0.1f" %chi2R,(plt_x,ylim[1]),color=color,ha='center')
    pars_jk,props_jk=yu.jackMA(fits)
    ens2mass_jk[ens]['mC']=mC_MA_jk=pars_jk[:,:1]
    
    pars_mean_MA,pars_err_MA,props=yu.jackMA2(fits)
    plt_y=pars_mean_MA[0]*yUnit; plt_yerr=pars_err_MA[0]*yUnit
    axs[irow,icol].fill_between([5.1,10],plt_y-plt_yerr,plt_y+plt_yerr,color=color,alpha=1)
    
    tmean,terr=yu.jackme(mC_MA_jk)
    plt_y=tmean*yUnit; plt_yerr=terr*yUnit
    axs[irow,icol].fill_between([0,5],plt_y-plt_yerr,plt_y+plt_yerr,color=color,alpha=1)

    fits=[]
    for fitmin in ens2fitmins_plt[ens]:
        color='b'
        pars_jk,chi2_jk,Ndof=ens2fit0_0c[ens][fitmin]
        chi2R=np.mean(chi2_jk)/Ndof
        if chi2R>100:
            continue
        fits.append([pars_jk,chi2_jk,Ndof])
        pars_mean,pars_err=yu.jackme(pars_jk)
        plt_x=fitmin*xUnit; plt_y=pars_mean[0]*yUnit; plt_yerr=pars_err[0]*yUnit
        axs[irow,icol].errorbar(plt_x,plt_y,plt_yerr,color=color)
        if chi2R>10 or fitmin%ens2chi2jump[ens] != 0:
            continue
        ylim=axs[irow,icol].get_ylim()
        axs[irow,icol].annotate("%0.1f" %chi2R,(plt_x,ylim[0]),color=color,ha='center')
    pars_jk,props_jk=yu.jackMA(fits)
    ens2mass_jk[ens]['m0_0c']=m0_0c_MA_jk=pars_jk[:,:1]
    
    pars_mean_MA,pars_err_MA,props=yu.jackMA2(fits)
    plt_y=pars_mean_MA[0]*yUnit; plt_yerr=pars_err_MA[0]*yUnit
    axs[irow,icol].fill_between([5.1,10],plt_y-plt_yerr,plt_y+plt_yerr,color=color,alpha=0.2)
    
    tmean,terr=yu.jackme(m0_0c_MA_jk)
    plt_y=tmean*yUnit; plt_yerr=terr*yUnit
    axs[irow,icol].fill_between([0,5],plt_y-plt_yerr,plt_y+plt_yerr,color=color,alpha=0.2)
    
    fits=[]
    for fitmin in ens2fitmins_plt[ens]:
        color='purple'
        pars_jk,chi2_jk,Ndof=ens2fit0[ens][fitmin]
        chi2R=np.mean(chi2_jk)/Ndof
        if chi2R>100:
            continue
        fits.append([pars_jk,chi2_jk,Ndof])
        pars_mean,pars_err=yu.jackme(pars_jk)
        plt_x=fitmin*xUnit; plt_y=pars_mean[0]*yUnit; plt_yerr=pars_err[0]*yUnit
        axs[irow,icol].errorbar(plt_x,plt_y,plt_yerr,color=color)
        if chi2R>10 or fitmin%ens2chi2jump[ens] != 0:
            continue
        ylim=axs[irow,icol].get_ylim()
        axs[irow,icol].annotate("%0.1f" %chi2R,(plt_x,ylim[0]+20),color=color,ha='center')
    pars_jk,props_jk=yu.jackMA(fits)
    ens2mass_jk[ens]['m0']=m0_MA_jk=pars_jk[:,:1]
    
    pars_mean_MA,pars_err_MA,props=yu.jackMA2(fits)
    plt_y=pars_mean_MA[0]*yUnit; plt_yerr=pars_err_MA[0]*yUnit
    axs[irow,icol].fill_between([5.1,10],plt_y-plt_yerr,plt_y+plt_yerr,color=color,alpha=0.2)
    
    tmean,terr=yu.jackme(m0_MA_jk)
    plt_y=tmean*yUnit; plt_yerr=terr*yUnit
    axs[irow,icol].fill_between([0,5],plt_y-plt_yerr,plt_y+plt_yerr,color=color,alpha=0.2)

axs[0,0].legend(fontsize=10,loc='upper center')

plt.tight_layout()
plt.savefig(f'fig/fits_MA0jack.pdf')
plt.close()

In [ ]:
# # test jackknife + modelAvg 2
# ens2fitmins_plt={'a':range(1,38), 'b':range(1,45), 'c':range(1,45), 'd':range(1,48)}
# ens2chi2jump={'a':3,'b':3,'c':3,'d':4}
# ens2mass_jk={}

# fig, axs = yu.getFigAxs(4,2,Lrow=4,Lcol=40,sharey=True,sharex='col')
# yu.addRowHeader(axs,[ens2label[ens] for ens in enss])
# yu.addColHeader(axs,['mEff','fits'])
    
# axs[0,0].set_xlim([0,6]); axs[0,1].set_xlim([0,3.7*2])
# axs[0,0].set_ylim([0,200])

# for irow,ens in enumerate(enss):
#     print(ens)
#     ens2mass_jk[ens]={}
#     mC_jk=ens2mC_jk[ens]
#     m0_jk=ens2m0_jk[ens]
#     m0d_jk=ens2m0d_jk[ens]
    
#     mC_mean,mC_err,mC_cov=yu.jackmec(mC_jk)
#     m0_mean,m0_err,m0_cov=yu.jackmec(m0_jk)
#     m0d_mean,m0d_err,m0d_cov=yu.jackmec(m0d_jk)
    
#     icol=0; xUnit=ens2a[ens]; yUnit=ens2aInv[ens]
#     tmean,terr=mC_mean,mC_err
#     axs[irow,icol].errorbar((np.arange(len(tmean)))*xUnit,tmean*yUnit,terr*yUnit,color='r',label='piC')
#     tmean,terr=m0_mean,m0_err
#     axs[irow,icol].errorbar((np.arange(len(tmean)))*xUnit,tmean*yUnit,terr*yUnit,color='b',label='pi0_r')
#     tmean,terr=m0d_mean,m0d_err
#     axs[irow,icol].errorbar((np.arange(len(tmean)))*xUnit,tmean*yUnit,terr*yUnit,color='purple',label='pi0_rd')
    
#     icol=1
#     fits=[]; fitLs=[]
#     for fitmin in ens2fitmins_plt[ens]:
#         color='r'
#         pars_jk,chi2_jk,Ndof=ens2fitC[ens][fitmin]
#         chi2R=np.mean(chi2_jk)/Ndof
#         if chi2R>100:
#             continue
#         fits.append([pars_jk,chi2_jk,Ndof]); fitLs.append(fitmin)
#         pars_mean,pars_err=yu.jackme(pars_jk)
#         plt_x=fitmin*xUnit; plt_y=pars_mean[0]*yUnit; plt_yerr=pars_err[0]*yUnit
#         axs[irow,icol].errorbar(plt_x,plt_y,plt_yerr,color=color)
#     pars_jk,props_jk=yu.jackMA(fits)
#     ens2mass_jk[ens]['mC']=mC_MA_jk=pars_jk[:,:1]
#     pars_mean_MA,pars_err_MA,props=yu.jackMA2(fits)
#     plt_y=pars_mean_MA[0]*yUnit; plt_yerr=pars_err_MA[0]*yUnit
#     axs[irow,icol].fill_between([5.1,10],plt_y-plt_yerr,plt_y+plt_yerr,color=color,alpha=1)
#     tmean,terr=yu.jackme(mC_MA_jk)
#     plt_y=tmean*yUnit; plt_yerr=terr*yUnit
#     axs[irow,icol].fill_between([0,5],plt_y-plt_yerr,plt_y+plt_yerr,color=color,alpha=1)
    
#     i=0
#     props,_=yu.jackme(props_jk)
#     for fitmin in ens2fitmins_plt[ens]:
#         pars_jk,chi2_jk,Ndof=ens2fitC[ens][fitmin]
#         chi2R=np.mean(chi2_jk)/Ndof
#         if chi2R>100:
#             continue
#         plt_x=fitmin*xUnit 
#         prop=props[i]
#         i+=1
#         if prop<0.05:
#             continue        
#         ylim=axs[irow,icol].get_ylim()
#         axs[irow,icol].annotate("%0.2f" %prop,(plt_x,ylim[1]),color=color,ha='center')

#     fits=[]
#     for fitmin in ens2fitmins_plt[ens]:
#         color='b'
#         pars_jk,chi2_jk,Ndof=ens2fit0_0c[ens][fitmin]
#         chi2R=np.mean(chi2_jk)/Ndof
#         if chi2R>100:
#             continue
#         fits.append([pars_jk,chi2_jk,Ndof])
#         pars_mean,pars_err=yu.jackme(pars_jk)
#         plt_x=fitmin*xUnit; plt_y=pars_mean[0]*yUnit; plt_yerr=pars_err[0]*yUnit
#         axs[irow,icol].errorbar(plt_x,plt_y,plt_yerr,color=color)
#     pars_jk,props_jk=yu.jackMA(fits)
#     ens2mass_jk[ens]['m0_0c']=m0_0c_MA_jk=pars_jk[:,:1]
#     pars_mean_MA,pars_err_MA,props=yu.jackMA2(fits)
#     plt_y=pars_mean_MA[0]*yUnit; plt_yerr=pars_err_MA[0]*yUnit
#     axs[irow,icol].fill_between([5.1,10],plt_y-plt_yerr,plt_y+plt_yerr,color=color,alpha=0.2)
#     tmean,terr=yu.jackme(m0_0c_MA_jk)
#     plt_y=tmean*yUnit; plt_yerr=terr*yUnit
#     axs[irow,icol].fill_between([0,5],plt_y-plt_yerr,plt_y+plt_yerr,color=color,alpha=0.2)
    
#     i=0
#     props,_=yu.jackme(props_jk)
#     for fitmin in ens2fitmins_plt[ens]:
#         pars_jk,chi2_jk,Ndof=ens2fitC[ens][fitmin]
#         chi2R=np.mean(chi2_jk)/Ndof
#         if chi2R>100:
#             continue
#         plt_x=fitmin*xUnit 
#         prop=props[i]
#         i+=1
#         if prop<0.05:
#             continue        
#         ylim=axs[irow,icol].get_ylim()
#         axs[irow,icol].annotate("%0.2f" %prop,(plt_x,ylim[0]),color=color,ha='center')
    
#     fits=[]
#     for fitmin in ens2fitmins_plt[ens]:
#         color='purple'
#         pars_jk,chi2_jk,Ndof=ens2fit0[ens][fitmin]
#         chi2R=np.mean(chi2_jk)/Ndof
#         if chi2R>100:
#             continue
#         fits.append([pars_jk,chi2_jk,Ndof])
#         pars_mean,pars_err=yu.jackme(pars_jk)
#         plt_x=fitmin*xUnit; plt_y=pars_mean[0]*yUnit; plt_yerr=pars_err[0]*yUnit
#         axs[irow,icol].errorbar(plt_x,plt_y,plt_yerr,color=color)
#     pars_jk,props_jk=yu.jackMA(fits)
#     ens2mass_jk[ens]['m0']=m0_MA_jk=pars_jk[:,:1]
#     pars_mean_MA,pars_err_MA,props=yu.jackMA2(fits)
#     plt_y=pars_mean_MA[0]*yUnit; plt_yerr=pars_err_MA[0]*yUnit
#     axs[irow,icol].fill_between([5.1,10],plt_y-plt_yerr,plt_y+plt_yerr,color=color,alpha=0.2)
#     tmean,terr=yu.jackme(m0_MA_jk)
#     plt_y=tmean*yUnit; plt_yerr=terr*yUnit
#     axs[irow,icol].fill_between([0,5],plt_y-plt_yerr,plt_y+plt_yerr,color=color,alpha=0.2)
    
#     i=0
#     props,_=yu.jackme(props_jk)
#     for fitmin in ens2fitmins_plt[ens]:
#         pars_jk,chi2_jk,Ndof=ens2fitC[ens][fitmin]
#         chi2R=np.mean(chi2_jk)/Ndof
#         if chi2R>100:
#             continue
#         plt_x=fitmin*xUnit 
#         prop=props[i]
#         print(i,fitmin,fitmin*xUnit,"%0.2f" %prop)
#         i+=1
#         if prop<0.05:
#             continue        
#         ylim=axs[irow,icol].get_ylim()
#         axs[irow,icol].annotate("%0.2f" %prop,(plt_x,ylim[0]+20),color=color,ha='center')

# axs[0,0].legend(fontsize=10,loc='upper center')

# plt.tight_layout()
# plt.savefig(f'fig/fits_MA0jack2.pdf')
# plt.close()